# Geohash + CID


Geohash functionality in Python [pygeohash](https://github.com/wdm0006/pygeohash)


In [ ]:
#!pip install pygeohash

In [5]:
import pygeohash as pgh
##load geojson
import geopandas as gpd
import pandas as pd
import os
import sys
sys.path.append("../")
from geohashtree.geohashtree import LiteTreeOffset
from geohashtree.filesystem import ipfs_add_feature

In [2]:
asset = "dc_restaurants"
geojson_path = f"../assets/{asset}.geojson"
index_path = f"../data/geohash_offset_{asset}/index"

## Index Preparation
using one large file + lite index tree with offset

In [3]:
offsettree = LiteTreeOffset()
offsettree.add_from_geojson(geojson_path)
offsettree.generate_tree_index(index_path)

 ../data/geohash_offset_dc_restaurants/index ../data/geohash_offset_dc_restaurants/index/ ../data/geohash_offset_dc_restaurants/index/.txt
d ../data/geohash_offset_dc_restaurants/index/ ../data/geohash_offset_dc_restaurants/index//d ../data/geohash_offset_dc_restaurants/index//d.txt
dq ../data/geohash_offset_dc_restaurants/index//d ../data/geohash_offset_dc_restaurants/index//d/dq ../data/geohash_offset_dc_restaurants/index//d/dq.txt
dqc ../data/geohash_offset_dc_restaurants/index//d/dq ../data/geohash_offset_dc_restaurants/index//d/dq/dqc ../data/geohash_offset_dc_restaurants/index//d/dq/dqc.txt
dqcm ../data/geohash_offset_dc_restaurants/index//d/dq/dqc ../data/geohash_offset_dc_restaurants/index//d/dq/dqc/dqcm ../data/geohash_offset_dc_restaurants/index//d/dq/dqc/dqcm.txt
dqcmc ../data/geohash_offset_dc_restaurants/index//d/dq/dqc/dqcm ../data/geohash_offset_dc_restaurants/index//d/dq/dqc/dqcm/dqcmc ../data/geohash_offset_dc_restaurants/index//d/dq/dqc/dqcm/dqcmc.txt
dqcmc4 ../data/g

## Onloading

In [6]:
ipfs_add_feature(geojson_path)

'bafybeia7jlkyzt22qgh66felldr322l5c7jdvm4xspr222f4usnd2ttbze'

In [8]:
cid = "bafybeia7jlkyzt22qgh66felldr322l5c7jdvm4xspr222f4usnd2ttbze"

In [7]:
import subprocess

In [9]:
of,lg = (6266126, 4005)

In [11]:
result = subprocess.run(['ipfs', 'cat', cid, "-o",str(of),"-l",str(lg)], stdout=subprocess.PIPE, check=True, text=True)

In [12]:
result.stdout

'{ "type": "Feature", "properties": { "full_id": "n5363946409", "osm_id": "5363946409", "osm_type": "node", "amenity": "restaurant", "atm": null, "opening_hours:bar": null, "survey:date": null, "brewery": null, "payment:cheque": null, "tourism": null, "artwork_type": null, "payment:notes": null, "diet:healthy": null, "diet:organic": null, "twitter": null, "opening_hours:lunch": null, "opening_hours:happy_hour": null, "opening_hours:dinner": null, "opening_date": null, "diet:kosher": null, "addr:place": null, "cocktails": null, "disused:name": null, "disused": null, "height": null, "vacant": null, "disused:shop": null, "organic": null, "payment:coins": null, "proposed:amenity": null, "name:zh": null, "karaoke": null, "old_name:vi": null, "contact:yelp": null, "alt_name:vi": null, "instagram": null, "name:eo": null, "proposed:shop": null, "proposed:name": null, "planned:amenity": null, "construction:amenity": null, "diet:meat": null, "payment:jcb": null, "website_1": null, "website:menu"

## Spatial query
Perform a spatial query and return a list of geohashes

In [13]:
geohashes = ['dqcnn', 'dqcjz', 'dqcjw', 'dqcjv', 'dqcnp', 'dqcjx', 'dqcjt', 'dqcnj']

## Feature Retrieval

In [15]:
offsettree.query(geohashes,index_path)

{'bafybeia7jlkyzt22qgh66felldr322l5c7jdvm4xspr222f4usnd2ttbze': ['(6033814, 3921)',
  '(10410040, 3946)',
  '(8394896, 4025)',
  '(7717359, 3931)',
  '(6816189, 3993)',
  '(2946205, 3982)',
  '(2384810, 3940)',
  '(4734973, 3938)',
  '(2289283, 3980)',
  '(630149, 3983)',
  '(9379417, 4009)',
  '(7382628, 3969)',
  '(5119506, 4127)',
  '(2400630, 3951)',
  '(8747952, 3930)',
  '(2380820, 3990)',
  '(6780433, 3929)',
  '(7510533, 3946)',
  '(2507853, 3946)',
  '(7837723, 3962)',
  '(746369, 3968)',
  '(850216, 3970)',
  '(4219721, 3961)',
  '(7526321, 4023)',
  '(1957920, 3973)',
  '(9331615, 3932)',
  '(4036035, 3966)',
  '(6157952, 4020)',
  '(2575846, 4045)',
  '(750337, 3982)',
  '(2285320, 3963)',
  '(1573024, 3965)',
  '(9612182, 3983)',
  '(5514109, 4018)',
  '(3724735, 4143)',
  '(2332943, 3961)',
  '(4167931, 3960)',
  '(3548567, 3940)',
  '(2090382, 4014)',
  '(8158335, 3942)',
  '(802386, 3972)',
  '(2408546, 3968)',
  '(8728281, 3933)',
  '(7894126, 4001)',
  '(7681762, 4027

In [16]:
offsettree.retrieve(geohashes,index_path)

330


,full_id,osm_id,osm_type,amenity,atm,opening_hours:bar,survey:date,brewery,payment:cheque,tourism,...,brand:wikidata,brand,addr:suite,addr:street,addr:state,addr:postcode,addr:housenumber,addr:city,entrance,geometry
0,n10863776715,10863776715,node,restaurant,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-77.05144 39.04216)
1,n10979494184,10979494184,node,restaurant,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-77.04775 39.04156)
2,n10979494185,10979494185,node,restaurant,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-77.04773 39.04166)
3,n10979539541,10979539541,node,restaurant,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-77.05213 39.04023)
4,n11113180851,11113180851,node,restaurant,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Wilkins Court,NaN,NaN,12307,NaN,NaN,POINT (-77.11267 39.05782)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,n10736129393,10736129393,node,restaurant,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,East West Highway,NaN,NaN,1302,NaN,NaN,POINT (-77.03224 38.99244)
325,n10748530279,10748530279,node,restaurant,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Summit Avenue,MD,20895,10525,Kensington,NaN,POINT (-77.07835 39.02892)
326,n10771881091,10771881091,node,restaurant,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Cordell Avenue,MD,20814,4931,Bethesda,NaN,POINT (-77.09908 38.98853)
327,n10773917564,10773917564,node,restaurant,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Georgia Avenue,MD,20906,12321,Silver Spring,NaN,POINT (-77.04955 39.05936)
